In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import os

In [2]:
# Mars News URL to scrape
url = 'https://redplanetscience.com/'

# Retrieve page with the requests module
html = requests.get(url)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html.text, 'html.parser')

In [25]:
# Examine the results and look for a ul with the class 'querylist'
result = soup.find('div', id="news",class_='container')

# Retrieve each list item
queries = result.find_all(class_='row')

In [24]:
for query in queries:
    print(query)

In [23]:
# Loop through returned queries
for query in queries:

    # Retrieve the query title
    title = query.find('div', class_='content_title')

    # Access the query's text content
    title_text = title.text

    # Access the query with CSS selectors
    preview = query.find('div', class_='article_teaser_body')    

    print('\n-----------------\n')
    print(title_text)
    print(preview)
